<a href="https://colab.research.google.com/github/Boluwahduro/Web_Scraping_GooglePlayStore/blob/main/Web_Scraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web Scraping Google Play Store

In [ ]:
#import necessary libaries

from google_play_scraper import app
from google_play_scraper import Sort, reviews_all
from numpy.random import default_rng
from time import sleep

Using news reports, data was scraped for Spotify top global markets across North America, Latin America, Asia and Africa. Countries representing the continents are United States, Canada, United Kingdon, Ireland, Sweden, Mexico, Spain, Phillipines, India, Nigeria, South Africa, Ghana, and Kenya

### Google Play Store Web Scraping- Includes saving the scraped data if error encountred and catch all exceptions

In [ ]:
#fecth all reviews for the specified countries from google API

def scrape_reviews(app_id, country_codes=['us', 'ca', 'uk', 'ie', 'se', 'mx', 'es', 'ph', 'id', 'ng', 'sa', 'gh','ke'], retries=3):
    # Initialize a DataFrame to store all reviews
    df_all = pd.DataFrame()

    # Create random generator
    rng = default_rng()

    # Loop through all country codes and fetch reviews
    for country in country_codes:
        for _ in range(retries):
            try:
                # generates a scalar [single] value greater than or equal to 1 but less than 5
                time_to_sleep = rng.uniform(1, 5)
                sleep(time_to_sleep)  # Add delay between requests

                result = reviews_all(
                    app_id,
                    sleep_milliseconds=0,  # 0 ms means no sleep.
                    lang='en',  # Language is English
                    country=country,  # Use current country
                    sort=Sort.NEWEST,  # Sort by newest
                )

                # Convert the result to a DataFrame and add a country column
                df = pd.DataFrame(result)
                df['country'] = country

                # Append the data to the master DataFrame
                df_all = pd.concat([df_all, df])

                # If my code runs to this point, it means I have successfully fetched the data, break the retry loop
                break

            except Exception as e:
                print(f'Error occurred when fetching reviews for {app_id} in {country}. Retrying...')
                print(f"Exception details: {e}")

        # Save the scraped data to a CSV file after each country
        df_all.to_csv('google_play_reviews.csv', index=False)

    return df_all

# Use the function
df_ggps = scrape_reviews('com.spotify.music')

In [ ]:
#df_ggplay = pd.read_csv(r"C:\Users\USER\Documents\UHull\Research Project\google_play_reviews.csv") #read file using lab
df_ggplay = pd.read_csv('/content/drive/My Drive/Project/google_play_reviews.csv') #using google colab

In [ ]:
# # Load data
# #df = pd.read_csv("google_play_reviews.csv")

# # Convert 'at' column to datetime
# df_ggplay['at'] = pd.to_datetime(df_ggplay['at'])

# # Get the distribution of reviews by year
# year_distribution = df_ggplay['at'].dt.year.value_counts()

# # Print the distribution
# print(year_distribution)


In [ ]:
df_ggplay.count() #before filter

reviewId                3444292
userName                3444277
userImage               3444292
content                 3444292
score                   3444292
thumbsUpCount           3444292
reviewCreatedVersion    3085965
at                      3444292
replyContent             209184
repliedAt                209184
appVersion              3085965
country                 3444292
dtype: int64

In [ ]:
print(df_ggplay.columns) #display columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion', 'country'],
      dtype='object')


In [ ]:
# Initialize the first chunk flag
first_chunk = True

# Initialize counters for the statistics
total_rows = 0
total_dup_content = 0
total_rows_after_dropping = 0

# Keep track of all unique pairs of 'userName' and 'content'
unique_pairs = set()

# Load the data in chunks and process each chunk
chunksize = 10 ** 6  # adjusted this value based on system's memory
for chunk in pd.read_csv(r"C:\Users\USER\Documents\UHull\Research Project\google_play_reviews.csv", chunksize=chunksize):
    # Specify the columns to keep
    chunk = chunk[['userName', 'at', 'content', 'appVersion', 'country']].copy()

    # Convert 'at' column to datetime and filter rows where year is 2022 or later
    chunk['at'] = pd.to_datetime(chunk['at'])
    chunk = chunk[chunk['at'].dt.year >= 2022]

    # Convert 'userName' and 'content' to lowercase
    chunk['userName'] = chunk['userName'].str.lower()
    chunk['content'] = chunk['content'].str.lower()

    # Update total_rows counter
    total_rows += chunk.shape[0]

    # Update total_dup_content counter
    total_dup_content += chunk.duplicated(subset=['userName', 'content']).sum()

    # Keep only the rows where the pair of 'userName' and 'content' is not in unique_pairs
    chunk = chunk[~chunk.apply(lambda row: (row['userName'], row['content']) in unique_pairs, axis=1)]

    # Add the pairs of 'userName' and 'content' from this chunk to unique_pairs
    unique_pairs.update(chunk.apply(lambda row: (row['userName'], row['content']), axis=1).tolist())

    # Update total_rows_after_dropping counter
    total_rows_after_dropping += chunk.shape[0]

    # Add 'source' column and rename columns
    chunk['source'] = 'GooglePlayStore'
    chunk.columns = ['userName', 'date', 'review', 'appVersion', 'country', 'source']

    # Save the chunk to a new CSV file, appending if it's not the first chunk
    if first_chunk:
        chunk.to_csv(r"C:\Users\USER\Documents\UHull\Research Project\google_play_reviews_filtered.csv", index=False)
        first_chunk = False
    else:
        chunk.to_csv(r"C:\Users\USER\Documents\UHull\Research Project\google_play_reviews_filtered.csv", mode='a', header=False, index=False)

# Print the initial number of rows, the number of duplicates in 'userName' and 'content', and the number of rows after dropping duplicates
# print("Initial number of rows:", total_rows)
# print("Number of duplicate 'userName' and 'content':", total_dup_content)
# print("Number of rows after dropping duplicates:", total_rows_after_dropping)


In [ ]:
#Read in the filtered dataset 2022 to the latest date in 2023
#df_ggplayf = pd.read_csv(r"C:\Users\USER\Documents\UHull\Research Project\Data\GooglePlayStore\google_play_reviews_filtered.csv")
df_ggplayf = pd.read_csv('/content/drive/My Drive/Project/Data/GooglePlayStore/google_play_reviews_filtered.csv') #using google colab


print(df_ggplayf.duplicated(subset=['userName', 'review']).sum())

235289


After performing chunk-wise duplicate removal process, there are still duplicates in the final DataFrame, df_ggplayf, when I load it into memory all at once. Attempted to load the chunks in batches previously.

A 'userName' and 'review' pair that appears in two different chunks would be considered unique in each individual chunk and would not be dropped.

In [ ]:
df_ggplayf.count()
df_ggplayf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340974 entries, 0 to 340973
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   userName    340968 non-null  object
 1   date        340974 non-null  object
 2   review      340974 non-null  object
 3   appVersion  310514 non-null  object
 4   country     340974 non-null  object
 5   source      340974 non-null  object
dtypes: object(6)
memory usage: 15.6+ MB


In [ ]:
# Drop duplicates based on 'userName' and 'review'
df_ggplayf.drop_duplicates(subset=['userName', 'review'], keep='first', inplace=True)

# Check the number of duplicates again
userName_review_dups_after = df_ggplayf.duplicated(subset=['userName', 'review']).sum()


#Dropping all duplicates in reviews as there are different usernames with the same review from different countries and saving this to a new csv file
#after dropping thwe duplicates- to be used for the sentiment analysis

# Drop duplicates based on 'review' column
df_ggplayf.drop_duplicates(subset=['review'], keep='first', inplace=True)

# Check the number of duplicates again
review_dups_after = df_ggplayf.duplicated(subset=['review']).sum()

print(f"Duplicates based on 'userName' and 'review' after dropping: {userName_review_dups_after}")
print(f"Duplicates based on 'review' after dropping: {review_dups_after}")


Duplicates based on 'userName' and 'review' after dropping: 0
Duplicates based on 'review' after dropping: 0


In [ ]:
# Check the number of rows left in the dataframe
df_ggplayf.shape[0]

104298

In [ ]:
#check for missing values
df_ggplayf.isnull().sum()

userName          2
date              0
review            0
appVersion    10428
country           0
source            0
dtype: int64

In [ ]:
# Set appVersion null to 'unknown'
df_ggplayf['appVersion'] = df_ggplayf['appVersion'].fillna('unknown')

# Change 'date' to datetime
df_ggplayf['date'] = pd.to_datetime(df_ggplayf['date'])


In [ ]:
df_ggplayf.head(2)

,userName,date,review,appVersion,country,source
0,jama bradfield,2023-06-28 01:13:53,"stoppage issues. since the last update, i've h...",8.8.46.529,us,GooglePlayStore
1,night pantheress,2023-06-24 18:59:07,the new update makes it nearly impossible to e...,8.8.44.527,us,GooglePlayStore


In [ ]:
# Check the number of duplicates again
print(df_ggplayf.duplicated(subset=['userName', 'review']).sum())
print(df_ggplayf.duplicated(subset=['review']).sum())

0
0


In [ ]:
# Drop 'userName' column
df_ggplayf = df_ggplayf.drop(columns='userName')

# Save the cleaned dataframe to a new CSV file
df_ggplayf.to_csv('google_play_reviews22.csv', index=False)

In [ ]:
df_ggplayreview = pd.read_csv('google_play_reviews22.csv') #cleaned and final data from googleplaystore

print(df_ggplayreview.columns)

Index(['date', 'review', 'appVersion', 'country', 'source'], dtype='object')


In [ ]:
print(df_ggplayreview['date'].min())
print(df_ggplayreview['date'].max())

2022-01-01 00:13:00
2023-07-04 16:18:38


In [ ]:
df_ggplayreview.head(5)

,date,review,appVersion,country,source
0,2023-06-28 01:13:53,"stoppage issues. since the last update, i've h...",8.8.46.529,us,GooglePlayStore
1,2023-06-24 18:59:07,the new update makes it nearly impossible to e...,8.8.44.527,us,GooglePlayStore
2,2023-06-25 19:12:44,it's the home page and the horrible playlists ...,8.8.46.529,us,GooglePlayStore
3,2023-06-20 12:52:10,using spotify for tv very 1.76.1 which updated...,8.8.44.527,us,GooglePlayStore
4,2023-06-24 10:45:51,i love spotify. it's a great app to listen to ...,8.8.46.529,us,GooglePlayStore
